#  Build a application of RAG test framework.

**Description** :

1. use vllm as llm service, consider the service is up in docker, and can be accessed by API,

2. use llamaindex as RAG framework, 

3. more then 2000 arxiv paper PDF data is parsed and save in markdown and json

4. write a jupyter notebook to run a RAG test framework. Explain the decision it made , for example why use certain chucking method

**Task**:

1. use a complex method to get relevant paper, 

    1.1 It requires vectorizing the parsed files with reasonable chucking method.

    1.2 It need to filter the paper which has little relation with the RAG method in large language model.

2. build a simple UI using 

**Test question**:

1. what is normal ways to optimizaed RAG system

2. what is the lastest RAG ideas that might be use to improve our RAG system 

In [ ]:
import os
import glob
import json
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
import gradio as gr

# Set up vLLM API endpoint (OpenAI-compatible)
os.environ["OPENAI_API_KEY"] = "EMPTY"  # vLLM does not require a real key
os.environ["OPENAI_API_BASE"] = "http://localhost:8889/v1"  # Change if your vLLM endpoint differs


# Define relevant keywords for filtering
RAG_KEYWORDS = [
    "retrieval-augmented", "RAG", "retrieval", "large language model", "LLM", "knowledge", "augmentation",
    "open-domain", "question answering", "document retrieval", "vector search", "semantic search"
]



## Filter the data：

using keyword matching is not a good idea, use llm as a reader to catagorize the summary

for example, within all RAG related paper, some paper introduce novel method to improve the RAG perfomance, some paper represent the usage of RAG in certain field, some paper happen to contain the keyword but has nothing to do with the technology.

now build a filter based on llm, the llm model is packed with vllm and access through http://localhost:8889/v1 in openai format.

data input:
all PDF papers are parsed and save into the D:\DataAnalysis\arxiv\output in seperate files. inside each file, a markdown file stores the parsed result, the summary is stored under the tag "# Abstract" 

task:
1 maintain a list of tags. these tags describe the catagories of the paper. For the start, the abover described catagoty can be used as first batch of tags. Tag1: introduce novel method to improve the RAG perfomance. Tag2: usage of RAG in certain field. Tag3: has nothing to do with the technology. More tags can be generated as the llm read more paper summaries. 

2 read all paper summary, get the llm judgement, save its tag into a file which contains the file name and the tag

3 mind that this work is related to later process to get access to the relevent paper

requirement:
1 using python and can be executed in jupyter notebook

2 asynchronous send query to vllm to reduce the time consumed

In [ ]:
import aiohttp
import asyncio
import re
import glob
import os
import json
import nest_asyncio
import pprint

nest_asyncio.apply()
# Define initial tags/categories
TAGS = [
    "introduce novel method to improve the RAG performance",
    "usage of RAG in certain field",
    "has nothing to do with the retrieval augmented generation in large language model application",
    "retrieval augmented generation only mention but not be discussed in detail",
    "cannot be catagorized into any mentioned tags, others"
]

new_tags = []
semaphore = asyncio.Semaphore(5)
output_filter_dir = r"D:\DataAnalysis\arxiv\output_filter"

# Helper: extract abstract from markdown
#def extract_abstract(md_text):
#    match = re.search(r"# Abstract\s*(.+?)(?:\n#|\Z)", md_text, re.DOTALL)
#    return match.group(1).strip() if match else ""

def extract_abstract(md_text):
    # 支持 "# Abstract", "Abstract.", "Abstract:", "## Abstract" 等多种写法
    match = re.search(
        r"(?:^|\n)\s{0,3}(?:#*\s*)?Abstract[\.:]?\s*(.+?)(?:\n#|\n[A-Z][^a-z]|$)",
        md_text, re.DOTALL | re.IGNORECASE
    )
    return match.group(1).strip() if match else ""
# Async function to call vLLM (OpenAI API format)
async def classify_abstract(session, abstract, tags):
    async with semaphore:
        prompt = (
            f"Given the following paper abstract, classify it into one of these categories:\n"
            f"{json.dumps(tags)}\n"
            f"Abstract:\n{abstract}\n"
            f"Respond with the most suitable category from the list. Do not output any other word other than the name of the type. \n /no_think"
        )
        payload = {
            "model": "/models/Qwen3-8B",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 50,
            "temperature": 0
        }
        async with session.post(
            "http://localhost:8889/v1/chat/completions",
            json=payload,
            timeout=60
        ) as resp:
            result = await resp.json()
            return result["choices"][0]["message"]["content"].strip()

# 批处理
def batch_iter(md_files, batch_size):
    """Yield successive batch_size-sized chunks from lst."""
    for i in range(0, len(md_files), batch_size):
        yield md_files[i:i + batch_size]

async def filter_and_tag_papers_in_batches(md_files, tags, batch_size=20):
    results = []
    with open("paper_tags.jsonl", "a", encoding="utf-8") as f1:  # 以追加模式打开
        async with aiohttp.ClientSession() as session:
            for batch in batch_iter(md_files, batch_size):
                tasks = []
                for md_file in batch:
                    with open(md_file, "r", encoding="utf-8") as f:
                        md_text = f.read()
                    abstract = extract_abstract(md_text)
                    if not abstract:
                        print(f"[SKIP] No abstract in: {md_file}")
                        continue
                    tasks.append(classify_abstract(session, abstract, tags))
                # gather this batch
                tag_results = await asyncio.gather(*tasks)
                print(f"Processed batch: {batch[0]} ... ({len(batch)} files)")
                for md_file, tag in zip(batch, tag_results):
                    # 去除 <think> 标签
                    if isinstance(tag, str):
                        tag = tag.replace("<think>\n\n</think>\n\n", "").strip()
                    result = {"file": md_file, "tag": tag}
                    results.append(result)
                    # 如果是无关论文，立即移动文件夹
                    if tag == "has nothing to do with the retrieval-augmented generation in large language model application":
                        parent_dir = os.path.dirname(md_file)
                        dest_dir = os.path.join(output_filter_dir, os.path.basename(parent_dir))
                        if not os.path.exists(dest_dir):
                            print(f"Moving {parent_dir} -> {dest_dir}")
                            shutil.move(parent_dir, dest_dir)
                            print(f"Moving: {batch[0]} ... ({len(batch)} files)")
                    f1.write(json.dumps(result, ensure_ascii=False) + "\n")  # 追加写入一行
                    
    return results


# Main async filter function 非批处理（会有奇怪错误）
async def filter_and_tag_papers(md_files, tags):
    results = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for md_file in md_files:
            with open(md_file, "r", encoding="utf-8") as f:
                md_text = f.read()
            abstract = extract_abstract(md_text)
            if not abstract:
                print(f"[SKIP] No abstract in: {md_file}")
                continue
            tasks.append(classify_abstract(session, abstract, tags))
        tag_results = await asyncio.gather(*tasks)
        for md_file, tag in zip(md_files, tag_results):
            results.append({"file": md_file, "tag": tag})
    return results

# Gather all markdown files
md_files = glob.glob(r"D:\DataAnalysis\arxiv\output\*\*.md")

# Run the async filter and save results
#tagged_results = await filter_and_tag_papers(md_files, TAGS)

tagged_results = await filter_and_tag_papers_in_batches(md_files, TAGS, 40)



# Save to file for later use
#with open("paper_tags.json", "w", encoding="utf-8") as f:
#    json.dump(tagged_results, f, indent=2, ensure_ascii=False)

#for result in tagged_results:
#    print(f"Tagged {len(tagged_results)} papers. Example:", result)



Processed batch: D:\DataAnalysis\arxiv\output\1007.1378v3\1007.1378v3.md ... (40 files)
Processed batch: D:\DataAnalysis\arxiv\output\1007.1378v3\1007.1378v3.md ... (40 files)
[SKIP] No abstract in: D:\DataAnalysis\arxiv\output\2308.10462v3\2308.10462v3.md
Processed batch: D:\DataAnalysis\arxiv\output\2308.00479v1\2308.00479v1.md ... (40 files)
Processed batch: D:\DataAnalysis\arxiv\output\2308.00479v1\2308.00479v1.md ... (40 files)
[SKIP] No abstract in: D:\DataAnalysis\arxiv\output\2312.17264v1\2312.17264v1.md
Processed batch: D:\DataAnalysis\arxiv\output\2311.09114v2\2311.09114v2.md ... (40 files)
Processed batch: D:\DataAnalysis\arxiv\output\2311.09114v2\2311.09114v2.md ... (40 files)
[SKIP] No abstract in: D:\DataAnalysis\arxiv\output\2401.15378v5\2401.15378v5.md
[SKIP] No abstract in: D:\DataAnalysis\arxiv\output\2401.17043v3\2401.17043v3.md
[SKIP] No abstract in: D:\DataAnalysis\arxiv\output\2401.17645v1\2401.17645v1.md
[SKIP] No abstract in: D:\DataAnalysis\arxiv\output\2402.00

In [7]:
dir=r"D:\DataAnalysis\arxiv\output\a\a.md"

dest_dir = os.path.join(output_filter_dir, os.path.basename(os.path.dirname(dir)))
dest_dir

'D:\\DataAnalysis\\arxiv\\output_filter\\a'

In [ ]:
parent_dir = os.path.dirname(md_file)
dest_dir = os.path.join(output_filter_dir, os.path.basename(parent_dir))
if not os.path.exists(dest_dir):
    print(f"Moving {parent_dir} -> {dest_dir}")
    shutil.move(parent_dir, dest_dir)
    print(f"Moving: {batch[0]} ... ({len(batch)} files)")


## 2. Chunking and Vectorization Strategy

local embedding model：Qwen/Qwen3-Embedding-8B

data input:
all PDF papers are parsed and save into the D:\DataAnalysis\arxiv\output in seperate files. inside each file, a markdown file stores the parsed result, the summary is stored under the tag "# Abstract" 

task:
1 maintain a list of tags. these tags describe the catagories of the paper. For the start, the abover described catagoty can be used as first batch of tags. Tag1: introduce novel method to improve the RAG perfomance. Tag2: usage of RAG in certain field. Tag3: has nothing to do with the technology. More tags can be generated as the llm read more paper summaries. 

2 read all paper summary, get the llm judgement, save its tag into a file which contains the file name and the tag

3 mind that this work is related to later process to get access to the relevent paper

requirement:
1 using python and can be executed in jupyter notebook

2 asynchronous send query to vllm to reduce the time consumed

functions


In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load model directly

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-8B")
model = AutoModel.from_pretrained("Qwen/Qwen3-Embedding-8B")

# Example: encode a batch of texts and get embeddings
def get_hf_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)
        # Use the last hidden state as embeddings (mean pooling)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example usage:
# embeddings = get_hf_embeddings(["Retrieval-augmented generation", "Large language models"])


# Build the vector index with chunking
embed_model = OpenAIEmbedding(model="text-embedding-3-small")  # Use vLLM's embedding endpoint if available

# Create the index (this may take a while for 2000+ papers)
index = VectorStoreIndex.from_documents(
    relevant_docs,
    embed_model=embed_model,
    vector_store=FaissVectorStore()
)

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)  # Use vLLM's API

query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=5,  # Retrieve top 5 relevant chunks
    response_mode="compact"
)


In [ ]:
## 4. Simple UI for RAG Testing
def rag_query(question):
    response = query_engine.query(question)
    return str(response)

demo = gr.Interface(
    fn=rag_query,
    inputs=gr.Textbox(label="Enter your RAG question"),
    outputs=gr.Textbox(label="RAG Answer"),
    title="RAG Test Framework (vLLM + LlamaIndex)",
    description="Ask questions about RAG or LLMs. The system retrieves from filtered arXiv papers and generates answers using vLLM."
)

# Uncomment to launch the UI
# demo.launch(share=True)
## 5. Test Questions

test_questions = [
    "What is normal ways to optimizaed RAG system?",
    "What is the lastest RAG ideas that might be use to improve our RAG system?"
]
for q in test_questions:
    print(f"Q: {q}")
    print("A:", rag_query(q))
    print("="*60)

## 6. Summary of Decisions

#- **Chunking:** Used heading/paragraph-based chunking to preserve academic structure and semantic boundaries.
#- **Filtering:** Only included papers with strong relevance to RAG/LLM, improving retrieval quality and speed.
#- **Vectorization:** Used OpenAI-compatible embeddings via vLLM for local, fast, and cost-effective embedding.
#- **UI:** Provided a simple Gradio interface for interactive testing.

#This framework can be extended with more advanced chunking, reranking, or hybrid retrieval methods as new RAG research emerges.


In [4]:
import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery:{query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'

queries = [
    get_detailed_instruct(task, 'What is the capital of China?'),
    get_detailed_instruct(task, 'Explain gravity')
]
# No need to add instruction for retrieval documents
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun."
]
input_texts = queries + documents
local_model_path = "D:/vllm_yml/models/Qwen3-Embedding-8B"


tokenizer = AutoTokenizer.from_pretrained(local_model_path, padding_side='left')
model = AutoModel.from_pretrained(local_model_path)

# We recommend enabling flash_attention_2 for better acceleration and memory saving.
# model = AutoModel.from_pretrained('Qwen/Qwen3-Embedding-8B', attn_implementation="flash_attention_2", torch_dtype=torch.float16).cuda()

max_length = 8192

# Tokenize the input texts
batch_dict = tokenizer(
    input_texts,
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt",
)
batch_dict.to(model.device)
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T)
print(scores.tolist())
# [[0.7493016123771667, 0.0750647559762001], [0.08795969933271408, 0.6318399906158447]]



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[[0.7493017911911011, 0.07506479322910309], [0.08795973658561707, 0.6318399906158447]]


In [8]:
import torch

#  
print(model)
print(tokenizer)

# simple inut 
test_text = "Hello world"
inputs = tokenizer(test_text, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)
    print(torch.isnan(outputs.last_hidden_state).any())  # check if nan

Qwen3Model(
  (embed_tokens): Embedding(151665, 4096)
  (layers): ModuleList(
    (0-35): 36 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
        (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
        (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
    )
  )
  (norm): Qwen3RMSNorm((

In [1]:
import torch

model.eval()  # 

test_text = "Hello world"
inputs = tokenizer(test_text, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

print(inputs)  #  

with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)
    print(torch.isnan(outputs.last_hidden_state).any())  # check if nan

NameError: name 'model' is not defined

In [5]:
inputs = tokenizer(test_text, return_tensors="pt")
inputs = {k: v.to(model.device) for k, v in inputs.items()}

inputs

{'input_ids': tensor([[  9707,   1879, 151643]]),
 'attention_mask': tensor([[1, 1, 1]])}

In [7]:
# Requires transformers>=4.51.0

import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]


def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery:{query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'

queries = [
    get_detailed_instruct(task, 'What is the capital of China?'),
    get_detailed_instruct(task, 'Explain gravity')
]
# No need to add instruction for retrieval documents
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun."
]
input_texts = queries + documents
local_model_path = "D:/vllm_yml/models/Qwen3-Embedding-8B"


tokenizer = AutoTokenizer.from_pretrained(local_model_path, padding_side='left')
model = AutoModel.from_pretrained(local_model_path)



# We recommend enabling flash_attention_2 for better acceleration and memory saving.
# model = AutoModel.from_pretrained('Qwen/Qwen3-Embedding-8B', attn_implementation="flash_attention_2", torch_dtype=torch.float16).cuda()

max_length = 8192

# Tokenize the input texts
batch_dict = tokenizer(
    input_texts,
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt",
)
batch_dict.to(model.device)
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T)
print(scores.tolist())
# [[0.7493016123771667, 0.0750647559762001], [0.08795969933271408, 0.6318399906158447]]


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[[0.7493017911911011, 0.07506479322910309], [0.08795973658561707, 0.6318399906158447]]


In [4]:
from sentence_transformers import SentenceTransformer
local_model_path = "D:/vllm_yml/models/Qwen3-Embedding-8B"

# Load the model
model = SentenceTransformer(local_model_path)

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-8B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )

# The queries and documents to embed
task = 'Given a web search query, retrieve relevant passages that answer the query'
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery:{query}'

queries = [
    get_detailed_instruct(task, 'What is the capital of China?'),
    get_detailed_instruct(task, 'Explain gravity')
]
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun."
]


# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)
# tensor([[0.7493, 0.0751],
#         [0.0880, 0.6318]])



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tensor([[0.7526, 0.0758],
        [0.0849, 0.6434]])


In [6]:
document_embeddings

array([[-0.01385963,  0.05234203, -0.00317804, ..., -0.00441788,
         0.00941126,  0.01125732],
       [ 0.0389513 ,  0.01941089, -0.01117104, ..., -0.00961749,
         0.00640514, -0.00067327]], shape=(2, 4096), dtype=float32)